In [3]:
import pandas as pd
import requests
from scrapy import Selector
from lxml import html
from lxml import etree

In [4]:
#Read in business names and links, scraped manually by searching SoS site
df = pd.read_csv('biz_names_and_real_links.csv')

In [35]:
#Loops to append values to separate lists
#All are to be concatenated into a dataframe after FOR loops
#If statements account for differences in SoS page structure

#Initialize lists to hold scraped variables
status=[]
org_type=[]
address=[]
filing_date=[]
owner_clerk=[]

id=0
for x in df['real_urls']:

    sel = Selector(text = requests.get(df['real_urls'][id]).content)
    
    if df['Type'][id] == 'MARK':
        status.append(sel.xpath('//table//b[contains(text(),"Status")]//ancestor::tr[1]/following::tr[1]/td[2]/text()').get())
        org_type.append(sel.xpath('//table//b[contains(text(),"Owner Type")]//ancestor::tr[1]/following::tr[1]/td[5]/text()').get())
        address.append(sel.xpath('//table//b[contains(text(),"Owner")]//ancestor::tr[1]/following::tr[3]/td/text()[last()-1] | //table//b[contains(text(),"Owner")]//ancestor::tr[1]/following::tr[3]/td/text()[last()]').extract())
        owner_clerk.append(sel.xpath('//table//b[contains(text(),"Owner")]//ancestor::tr[1]/following::tr[3]/td/text()[1]').extract())
        filing_date.append(sel.xpath('//table//b[contains(text(),"Filing Date")]//ancestor::tr[1]/following::tr[1]/td[2]/text()').extract())
    else: 
        status.append(sel.xpath('//table//b[contains(text(),"Status")]//ancestor::tr[1]/following::tr[1]/td[4]').get())
        org_type.append(sel.xpath('//table//b[contains(text(),"Filing Type")]//ancestor::tr[1]/following::tr[1]/td[3]').get())
        address.append(sel.xpath('//table//b[contains(text(),"Clerk")]//ancestor::tr[1]/following::tr[1]/td/text()[following::br][last()-1] | //table//b[contains(text(),"Clerk")]//ancestor::tr[1]/following::tr[1]/td/text()[following::br][last()]').extract())
        owner_clerk.append(sel.xpath('//table//b[contains(text(),"Clerk")]//ancestor::tr[1]/following::tr[1]/td/text()[following::br][1]').extract())
        filing_date.append(sel.xpath('//table//b[contains(text(),"Filing Date")]//ancestor::tr[1]/following::tr[1]/td[1]/text()').extract())
    id+=1

In [32]:
id

611

In [38]:
#Make lists to dataframe and concatenate with df
all_biz = pd.concat([df,pd.DataFrame({'status':status
                                      ,'owner/org_type':org_type
                                      ,'address':address
                                      ,'mark_owner_or_clerk':owner_clerk
                                      ,'filing_date':filing_date})], axis=1)
all_biz

,Index,Name,Type,real_urls,status,owner/org_type,address,mark_owner_or_clerk,filing_date
0,1,MAINE-LY A STITCH IN TIME,MARK,https://icrs.informe.org/nei-sos-icrs/ICRS?Mar...,EXPIRED,INDIVIDUAL,"[RR 2, BOX 1008 , \nPHILLIPS, ME 04966 \n\t\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMARY O....,[06/08/1993]
1,2,MAINE-LY ACCOUNTING INCORPORATED,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> ADMINISTRATIVELY SUSPENDED</td>,<td>BUSINESS CORPORATION</td>,"[\n162 U.S. ROUTE 1 BOX 4 , \nSCARBOROUGH, ME...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCRAIG R...,[]
2,3,"MAINE-LY ACTION RENTALS, INC.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> DISSOLVED</td>,<td>BUSINESS CORPORATION</td>,"[\n19 MAIN STREET , \nHARRISON, ME 04040 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tTHOMAS ...,[]
3,4,MAINE-LY AIRBORNE LLC,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> ADMINISTRATIVELY DISSOLVED</td>,<td>LIMITED LIABILITY COMPANY (DOMESTIC)</td>,"[\n130 LAKE STREET , \nAUBURN, ME 04210 \n\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tJAMES J...,[]
4,5,MAINE-LY AMISH,ASSUMED,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> GOOD STANDING</td>,<td>BUSINESS CORPORATION</td>,"[ SUITE 200, \n PORTLAND, ME 04101 \n\t\t\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tEZEKIEL...,[]
5,6,MAINE-LY APPLES,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> GOOD STANDING</td>,<td>BUSINESS CORPORATION</td>,"[\nP.O. BOX 205 , \nPITTSFIELD, ME 04967 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tALFRED ...,[]
6,7,"MAINE-LY AQUARIUMS, LLC",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> ADMINISTRATIVELY DISSOLVED</td>,<td>LIMITED LIABILITY COMPANY (DOMESTIC)</td>,"[\n41 POND VIEW DR , \nLEBANON, ME 04027 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tERIC DU...,[]
7,8,MAINE-LY ASIAN IMPORTS INC.,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> DISSOLVED</td>,<td>BUSINESS CORPORATION</td>,"[\n10 BRACKETT STREET , \nBIDDEFORD, ME 04005 ...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tDAVID A...,[]
8,9,MAINELY A MEMORY,ASSUMED,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> CANCELLED</td>,<td>LIMITED LIABILITY COMPANY (DOMESTIC)</td>,"[ , \n LEWISTON, ME 04243 1081\n\t\t\t\t\t\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tPAUL R ...,[]
9,10,"MAINELY ABODES, L.L.C.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,<td> ADMINISTRATIVELY DISSOLVED</td>,<td>LIMITED LIABILITY COMPANY (DOMESTIC)</td>,"[\nPO BOX 74 , \nSULLIVAN, ME 04664 \n\t\t\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMARLY S...,[]


In [56]:
#Cleaning HTML out of data
def trim_all_columns(df):
    """
    Trim whitespace from all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

df = trim_all_columns(df)

#Eliminate <td> tags
replace_dict = {'<td>':''
               ,'</td>':''
               ,r'\s+|\\n'}
all_biz.replace(replace_dict,regex=True)

,Index,Name,Type,real_urls,status,owner/org_type,address,mark_owner_or_clerk,filing_date
0,1,MAINE-LY A STITCH IN TIME,MARK,https://icrs.informe.org/nei-sos-icrs/ICRS?Mar...,EXPIRED,INDIVIDUAL,"[RR 2, BOX 1008 , \nPHILLIPS, ME 04966 \n\t\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMARY O....,[06/08/1993]
1,2,MAINE-LY ACCOUNTING INCORPORATED,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY SUSPENDED,BUSINESS CORPORATION,"[\n162 U.S. ROUTE 1 BOX 4 , \nSCARBOROUGH, ME...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tCRAIG R...,[]
2,3,"MAINE-LY ACTION RENTALS, INC.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,DISSOLVED,BUSINESS CORPORATION,"[\n19 MAIN STREET , \nHARRISON, ME 04040 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tTHOMAS ...,[]
3,4,MAINE-LY AIRBORNE LLC,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"[\n130 LAKE STREET , \nAUBURN, ME 04210 \n\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tJAMES J...,[]
4,5,MAINE-LY AMISH,ASSUMED,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,GOOD STANDING,BUSINESS CORPORATION,"[ SUITE 200, \n PORTLAND, ME 04101 \n\t\t\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tEZEKIEL...,[]
5,6,MAINE-LY APPLES,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,GOOD STANDING,BUSINESS CORPORATION,"[\nP.O. BOX 205 , \nPITTSFIELD, ME 04967 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tALFRED ...,[]
6,7,"MAINE-LY AQUARIUMS, LLC",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"[\n41 POND VIEW DR , \nLEBANON, ME 04027 \n\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tERIC DU...,[]
7,8,MAINE-LY ASIAN IMPORTS INC.,LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,DISSOLVED,BUSINESS CORPORATION,"[\n10 BRACKETT STREET , \nBIDDEFORD, ME 04005 ...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tDAVID A...,[]
8,9,MAINELY A MEMORY,ASSUMED,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,CANCELLED,LIMITED LIABILITY COMPANY (DOMESTIC),"[ , \n LEWISTON, ME 04243 1081\n\t\t\t\t\t\t\...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tPAUL R ...,[]
9,10,"MAINELY ABODES, L.L.C.",LEGAL,https://icrs.informe.org/nei-sos-icrs/ICRS?Cor...,ADMINISTRATIVELY DISSOLVED,LIMITED LIABILITY COMPANY (DOMESTIC),"[\nPO BOX 74 , \nSULLIVAN, ME 04664 \n\t\t\t\t...",[\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\tMARLY S...,[]


In [30]:
#OUTPUT SANDBOX
all_biz.to_csv('mainely_businesses_scraped.csv')

In [473]:
selm = Selector(text = requests.get('https://icrs.informe.org/nei-sos-icrs/ICRS?MarkSumm=20080031+M').content)

##MARK VALUES
#Mark STATUS XPath
print('Status: ' + selm.xpath('//table[1]/tr[5]/td[2]/text()').get()
)

#Mark OWNER XPath
print('Owner Type: ' + selm.xpath('//table[1]/tr[7]/td[5]/text()').get()
)

#Mark FILING DATE XPath
print('Filing Date: ' + selm.xpath('//table[1]/tr[7]/td[2]/text()').get()
)

#OWNER NAME
print('Owner Name: ' + selm.xpath('//table[1]/tr/td[1]/text()[following-sibling::br]').get()
)

#OWNER ADDRESS
print('Owner Address: ' + selm.xpath('//table[1]/tr/td[1]/text()[preceding-sibling::br and following::br]').get() 
      + selm.xpath('//table[1]/tr[9]/td[1]/text()[last()]').get()
)

#//div[@id="population"]/text()[preceding-sibling::br and following-sibling::br]

Status: EXPIRED
Owner Type: PARTNERSHIP
Filing Date: 07/30/2007
Owner Name: 
								
									KEVIN A. WARD
Owner Address: PAMELA J. WARD 
NORRIDGEWOCK, ME 04957 
								
								


In [ ]:
#LEGAL/ASSUMED XPaths
seld = Selector(text = requests.get('https://icrs.informe.org/nei-sos-icrs/ICRS?CorpSumm=19840005+D').content)

#Legal STATUS XPath 
print('Status: ' + seld.xpath('//table[1]/tr[5]/td[4]').get()
)

#Legal FILING TYPE XPath
print('Owner Type: ' + seld.xpath('//table[1]/tr[5]/td[3]').get()
)

#Legal ADDRESS XPath
print(seld.xpath('//table//b[contains(text(),"Clerk")]//ancestor::tr[1]/following::tr[1]/td/text()[following::br][position()>1]')
                #tbody/tr[3]/td/table/tbody/tr[11]/td
                .extract())

In [322]:
#IF LOGIC SANDBOX
selx = Selector(text = requests.get('https://icrs.informe.org/nei-sos-icrs/ICRS?CorpSumm=20121980DC').content)

if selx.xpath('//table[1]/tr[9]/td[1]/text()[preceding-sibling::br and following::br]').get() is None: 
    selx.xpath('//table[1]/tr[12]/td[1]/text()[preceding-sibling::br and following::br]').get() + selx.xpath('//table[1]/tr[12]/td[1]/text()[last()]').get()
else:
    selx.xpath('//table[1]/tr[9]/td[1]/text()[preceding-sibling::br and following::br]').get() + selx.xpath('//table[1]/tr[9]/td[1]/text()[last()]').get()

In [25]:
#XPATH SANDBOX
selx = Selector(text = requests.get('https://icrs.informe.org/nei-sos-icrs/ICRS?MarkSumm=19930285+M').content)
print(selx.xpath('//table//b[contains(text(),"Filing Date")]//ancestor::tr[1]/following::tr[1]/td[2]/text()').extract())


['06/08/1993']
